<a href="https://colab.research.google.com/github/Zhenriquee/ANALISE_OPERADORAS/blob/main/Extra%C3%A7%C3%A3o_e_Tratamento_dos_Dados_ANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coleta e Tratamento de Dados da ANS

## Objetivo:
Realizar a extração dos dados da ANS com a finalidade de analisar o posicionamento da Unimed Caruaru com Relação as outras Operadoras, no final iremos armazenar esses dados em um arquivo .db e Utilizar o streamlit para projeção desses dados.

## Links Utilizados para Extração


*   [Qtd. Beneficiarios por Trimestre](https://dadosabertos.ans.gov.br/FTP/Base_de_dados/Microdados/dados_dbc/beneficiarios/operadoras/)
*   [Demonstração Contabeis](https://dadosabertos.ans.gov.br/FTP/PDA/demonstracoes_contabeis/)



### Bibliotecas Utilizadas

In [ ]:
%pip install datasus_dbc dbfread

In [ ]:
import os
import requests
import sqlite3
import pandas as pd
import uuid
import re
import io
import zipfile
import unicodedata
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datasus_dbc import decompress
from dbfread import DBF
from concurrent.futures import ProcessPoolExecutor, as_completed

## Inicio Extração e Tratamento Qtd. Beneficiarios por Trimestre



In [ ]:
# --- FUNÇÕES AUXILIARES (Worker) ---
# Precisam estar fora da classe para o multiprocessing funcionar bem no Windows

def _gerar_chave_trimestre(id_cmpt):
    try:
        s_cmpt = str(id_cmpt).strip()
        if len(s_cmpt) < 6: return None
        ano = s_cmpt[:4]
        mes = int(s_cmpt[4:6])
        trimestre = (mes - 1) // 3 + 1
        return f"{ano}-T{trimestre}"
    except:
        return None

def processar_arquivo_worker(link):
    """
    Função isolada que roda em um núcleo separado da CPU.
    Baixa, Converte, Filtra e Agrupa. Retorna um DataFrame pronto (ou None).
    """
    nome_arquivo = link.split('/')[-1]

    # Gera nomes únicos para evitar colisão entre processos
    id_unico = str(uuid.uuid4())
    temp_dbc = f"temp_{id_unico}.dbc"
    temp_dbf = f"temp_{id_unico}.dbf"

    colunas_desejadas = ['ID_CMPT', 'CD_OPERADO', 'NR_BENEF_T']
    resultado_df = None

    try:
        # 1. Download
        r = requests.get(link, stream=True, timeout=30)
        with open(temp_dbc, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

        # 2. Descompressão
        decompress(temp_dbc, temp_dbf)

        # 3. Leitura e Pandas
        table = DBF(temp_dbf, encoding='iso-8859-1', load=True)
        df = pd.DataFrame(iter(table))

        if not df.empty:
            # Verifica colunas
            if all(col in df.columns for col in colunas_desejadas):
                df = df[colunas_desejadas].copy()
                df['NR_BENEF_T'] = pd.to_numeric(df['NR_BENEF_T'], errors='coerce').fillna(0)

                # Agrupa e Soma (Reduzindo drasticamente o tamanho dos dados antes de retornar)
                df_agrupado = df.groupby(['ID_CMPT', 'CD_OPERADO'], as_index=False)['NR_BENEF_T'].sum()

                # Cria chave Trimestre
                df_agrupado['ID_TRIMESTRE'] = df_agrupado['ID_CMPT'].apply(_gerar_chave_trimestre)

                resultado_df = df_agrupado
            else:
                print(f"   [Worker] Ignorado {nome_arquivo}: Colunas ausentes.")

    except Exception as e:
        print(f"   [Worker] Erro em {nome_arquivo}: {e}")

    finally:
        # Limpeza rigorosa dos arquivos temporários deste processo
        if os.path.exists(temp_dbc): os.remove(temp_dbc)
        if os.path.exists(temp_dbf): os.remove(temp_dbf)

    return resultado_df

# --- CLASSE PRINCIPAL ---

class ImportadorANSParalelo:
    def __init__(self, db_path='dados_ans.db'):
        self.db_path = db_path

    def etapa_1_e_2_obter_links(self, url_origem):
        print(f"--- Mapeando arquivos em: {url_origem} ---")
        try:
            response = requests.get(url_origem)
            soup = BeautifulSoup(response.content, 'html.parser')
            links = []
            for link in soup.find_all('a'):
                href = link.get('href')
                if href and href.lower().endswith('.dbc'):
                    links.append(urljoin(url_origem, href))
            print(f"Total de arquivos encontrados: {len(links)}")
            return links
        except Exception as e:
            print(f"Erro ao obter links: {e}")
            return []

    def etapa_3_processar_paralelo(self, lista_links, tabela_destino='beneficiarios_agrupados', max_workers=4):
        """
        Gerencia os workers e grava no banco sequencialmente.
        """
        conn = sqlite3.connect(self.db_path)
        total = len(lista_links)
        processados = 0

        print(f"--- Iniciando Processamento Paralelo ({max_workers} Workers) ---")

        # Inicia o Pool de Processos
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            # Submete todas as tarefas
            # future_to_link é um dicionário para rastrear qual link pertence a qual tarefa
            future_to_link = {executor.submit(processar_arquivo_worker, link): link for link in lista_links}

            for future in as_completed(future_to_link):
                processados += 1
                link = future_to_link[future]
                nome = link.split('/')[-1]

                try:
                    df_resultado = future.result()

                    if df_resultado is not None and not df_resultado.empty:
                        # O momento da escrita no banco é sequencial (Thread Principal)
                        df_resultado.to_sql(tabela_destino, conn, if_exists='append', index=False)
                        print(f"[{processados}/{total}] Salvo: {nome} ({len(df_resultado)} registros)")
                    else:
                        print(f"[{processados}/{total}] Vazio/Ignorado: {nome}")

                except Exception as exc:
                    print(f"[{processados}/{total}] Falha ao recuperar resultado de {nome}: {exc}")

        conn.close()
        print("--- Processo Paralelo Finalizado ---")

# --- EXECUÇÃO ---

if __name__ == "__main__":
    # URL da ANS
    url_ans = "https://dadosabertos.ans.gov.br/FTP/Base_de_dados/Microdados/dados_dbc/beneficiarios/operadoras/"

    # Define quantos núcleos do processador você quer usar
    # Se seu PC for potente, pode aumentar. Geralmente 4 ou 8 é um bom número.
    WORKERS = os.cpu_count() or 4

    bot = ImportadorANSParalelo(db_path='base_ans_paralela.db')

    links = bot.etapa_1_e_2_obter_links(url_ans)

    if links:
        # Executa em paralelo
        bot.etapa_3_processar_paralelo(links, max_workers=WORKERS)

## Inicio Extração e Tratamento Demonstração Contabeis

In [ ]:
def _extrair_info_arquivo_worker(nome_arquivo, url_pasta_ano):
    """Lógica de Regex para descobrir Ano e Trimestre"""
    try:
        nome = nome_arquivo.lower()

        # 1. Extrair ANO da URL da pasta (Mais confiável)
        url_limpa = url_pasta_ano.rstrip('/')
        ano_pasta = url_limpa[-4:]

        if not ano_pasta.isdigit() or len(ano_pasta) != 4:
            match_ano = re.search(r'(20\d{2})', nome)
            if match_ano:
                ano_pasta = match_ano.group(1)
            else:
                return None

        # 2. Extrair TRIMESTRE
        # Regex 1: "1 trimestre", "1_trimestre", "1-trimestre"
        match_longo = re.search(r'([1-4])\s*[-_]?\s*(?:trimestre|tri)', nome)
        if match_longo: return f"{ano_pasta}-T{match_longo.group(1)}"

        # Regex 2: "1t", "4t"
        match_curto = re.search(r'([1-4])t', nome)
        if match_curto: return f"{ano_pasta}-T{match_curto.group(1)}"

        # Regex 3: "t1"
        match_inv = re.search(r't([1-4])', nome)
        if match_inv: return f"{ano_pasta}-T{match_inv.group(1)}"

        return None
    except:
        return None

def processar_zip_worker(args):
    """
    Função que roda em paralelo.
    Recebe uma tupla: (link_do_zip, url_da_pasta_ano)
    Retorna: DataFrame filtrado ou None
    """
    link_zip, url_pasta_ano = args
    nome_arquivo = link_zip.split('/')[-1]

    # Identifica a chave temporal
    chave_trimestre = _extrair_info_arquivo_worker(nome_arquivo, url_pasta_ano)

    if not chave_trimestre:
        return None

    try:
        # 1. Download (timeout aumentado para evitar quedas em arquivos grandes)
        r = requests.get(link_zip, timeout=120)

        # 2. Processamento em Memória
        with zipfile.ZipFile(io.BytesIO(r.content)) as z:
            csvs = [n for n in z.namelist() if n.lower().endswith('.csv')]
            if not csvs: return None

            nome_csv = csvs[0]
            with z.open(nome_csv) as f:
                # Lê tudo como string para não perder zeros a esquerda
                df = pd.read_csv(f, sep=';', encoding='iso-8859-1', dtype=str)

                # Limpa nomes das colunas (Upper + Strip)
                df.columns = [c.upper().strip() for c in df.columns]

                # Verifica se tem a coluna alvo
                if 'CD_CONTA_CONTABIL' in df.columns:
                    # FILTRA CONTA 31
                    df_filtrado = df[df['CD_CONTA_CONTABIL'] == '31'].copy()

                    if not df_filtrado.empty:
                        # Adiciona a chave temporal
                        df_filtrado['ID_TRIMESTRE'] = chave_trimestre

                        # --- TRATAMENTO DE VALOR ---
                        if 'VL_SALDO_FINAL' in df_filtrado.columns:
                            df_filtrado['VL_SALDO_FINAL'] = df_filtrado['VL_SALDO_FINAL'].str.replace('.', '', regex=False)
                            df_filtrado['VL_SALDO_FINAL'] = df_filtrado['VL_SALDO_FINAL'].str.replace(',', '.', regex=False)
                            df_filtrado['VL_SALDO_FINAL'] = pd.to_numeric(df_filtrado['VL_SALDO_FINAL'], errors='coerce')

                        # --- NOVO: SANITIZAÇÃO DE COLUNAS (CORREÇÃO DO ERRO) ---
                        # Aqui definimos EXATAMENTE o que vai para o banco.
                        # Qualquer coluna extra (como VL_SALDO_INICIAL ou DT_CARGA) será ignorada.
                        colunas_finais = ['REG_ANS', 'CD_CONTA_CONTABIL', 'VL_SALDO_FINAL', 'ID_TRIMESTRE']

                        # Verifica quais dessas colunas existem no DF atual (para evitar erro se faltar alguma)
                        colunas_existentes = [c for c in colunas_finais if c in df_filtrado.columns]

                        # Retorna apenas as colunas limpas
                        return df_filtrado[colunas_existentes]

    except Exception as e:
        # Imprime erro mas não para o processo inteiro
        # print(f"   [Erro Worker] Falha em {nome_arquivo}: {e}")
        pass

    return None

# --- CLASSE PRINCIPAL ---

class ExtratorContabilParalelo:
    def __init__(self, db_path='dados_ans.db'):
        self.db_path = db_path
        self.url_base = "https://dadosabertos.ans.gov.br/FTP/PDA/demonstracoes_contabeis/"

    def _mapear_todos_arquivos(self):
        """Varre as pastas de anos e retorna uma lista de tuplas (url_zip, url_ano)"""
        print(f"--- Mapeando estrutura de pastas em: {self.url_base} ---")
        tarefas = []

        try:
            # 1. Pega pastas de Anos
            r = requests.get(self.url_base)
            soup = BeautifulSoup(r.content, 'html.parser')
            links_anos = []
            for link in soup.find_all('a'):
                href = link.get('href')
                if href and re.match(r'\d{4}/', href):
                    links_anos.append(urljoin(self.url_base, href))

            print(f"Anos encontrados: {len(links_anos)}. Buscando ZIPs dentro de cada ano...")

            # 2. Pega ZIPs dentro de cada Ano
            # (Poderíamos paralelizar isso também, mas é rápido o suficiente ser sequencial)
            for url_ano in links_anos:
                try:
                    r_ano = requests.get(url_ano)
                    soup_ano = BeautifulSoup(r_ano.content, 'html.parser')
                    for link in soup_ano.find_all('a'):
                        href = link.get('href')
                        if href and href.lower().endswith('.zip'):
                            full_link = urljoin(url_ano, href)
                            # Guardamos a tupla (Link do Arquivo, Link da Pasta do Ano)
                            tarefas.append((full_link, url_ano))
                except:
                    print(f"Erro ao ler pasta: {url_ano}")

        except Exception as e:
            print(f"Erro no mapeamento: {e}")

        return tarefas

    def executar(self, tabela_destino='demonstracoes_contabeis', max_workers=8):
        # 1. Mapeamento (Sequencial, mas rápido)
        lista_tarefas = self._mapear_todos_arquivos()
        total_arquivos = len(lista_tarefas)

        if total_arquivos == 0:
            print("Nenhum arquivo encontrado.")
            return

        print(f"--- Iniciando Download e Processamento de {total_arquivos} arquivos ---")
        print(f"--- Workers Ativos: {max_workers} ---")

        conn = sqlite3.connect(self.db_path)
        processados = 0
        sucessos = 0

        # 2. Processamento Paralelo
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            # Envia todas as tarefas
            future_to_url = {executor.submit(processar_zip_worker, tarefa): tarefa for tarefa in lista_tarefas}

            for future in as_completed(future_to_url):
                processados += 1
                link, _ = future_to_url[future]
                nome = link.split('/')[-1]

                try:
                    df_resultado = future.result()

                    if df_resultado is not None and not df_resultado.empty:
                        # 3. Escrita no Banco (Sequencial e Segura)
                        df_resultado.to_sql(tabela_destino, conn, if_exists='append', index=False)
                        sucessos += 1
                        trimestre = df_resultado['ID_TRIMESTRE'].iloc[0]
                        print(f"[{processados}/{total_arquivos}] SALVO: {trimestre} ({len(df_resultado)} linhas) -> {nome}")
                    else:
                        print(f"[{processados}/{total_arquivos}] Ignorado/Vazio: {nome}")

                except Exception as exc:
                    print(f"[{processados}/{total_arquivos}] Falha na tarefa {nome}: {exc}")

        conn.close()
        print(f"--- FIM. {sucessos} arquivos processados com sucesso. ---")

# --- EXECUÇÃO ---
if __name__ == "__main__":
    db_nome = 'base_ans_paralela.db'

    # Ajuste o número de workers conforme sua internet e CPU
    # 8 costuma ser um bom número. Se a internet cair, reduza para 4.
    extrator = ExtratorContabilParalelo(db_path=db_nome)
    extrator.executar(max_workers=8)

## Inicio Extração Dimensão Operadora


In [ ]:
class ImportadorCadop:
    def __init__(self, db_path, csv_path):
        self.db_path = db_path
        self.csv_path = csv_path

    def _limpar_nome_coluna(self, texto):
        """Limpa apenas o cabeçalho (nome da coluna) para o banco de dados"""
        txt = unicodedata.normalize('NFKD', str(texto)).encode('ASCII', 'ignore').decode('ASCII')
        return txt.strip().lower().replace(' ', '_').replace('.', '').replace('/', '')

    def processar_cadop(self, tabela_destino='dim_operadoras'):
        if not os.path.exists(self.csv_path):
            print(f"ERRO: Arquivo não encontrado: {self.csv_path}")
            return

        print(f"--- Atualizando Tabela CADOP (Correção de Acentos) ---")

        try:
            conn = sqlite3.connect(self.db_path)

            # 1. MUDANÇA CRÍTICA: encoding='utf-8-sig'
            # 'utf-8' resolve o problema do "SÃ£o".
            # O sufixo '-sig' é importante caso o arquivo tenha sido salvo pelo Excel (remove caracteres ocultos no início).
            # dtype=str: Lemos TUDO como texto para proteger CPNJ e Código ANS (zeros a esquerda).
            df = pd.read_csv(
                self.csv_path,
                sep=';',
                encoding='utf-8-sig',
                dtype=str
            )

            # 2. Padronizar nomes das colunas
            df.columns = [self._limpar_nome_coluna(c) for c in df.columns]

            # 3. TRATAMENTO INTELIGENTE DE COLUNAS
            # O usuário pediu para verificar string vs int.
            # Como carregamos tudo como 'str' para segurança, iteramos para limpar o texto.

            print("   Aplicando tratamento nas colunas de texto...")
            for coluna in df.columns:
                # Verifica se a coluna é do tipo objeto (string/texto)
                if df[coluna].dtype == 'object':
                    # .str.strip() remove espaços vazios no começo e fim que atrapalham SQL
                    # Ex: "São Paulo " vira "São Paulo"
                    df[coluna] = df[coluna].str.strip()

            print(f"   Colunas processadas: {list(df.columns[:5])} ...")

            # 4. Salvar no SQLite (DROP e CREATE com if_exists='replace')
            df.to_sql(tabela_destino, conn, if_exists='replace', index=False)

            # 5. Recriar Índice
            if 'registro_ans' in df.columns:
                cursor = conn.cursor()
                cursor.execute(f'CREATE INDEX IF NOT EXISTS idx_cadop_reg ON {tabela_destino}(registro_ans);')
                conn.commit()

            conn.close()

            # Validação visual
            exemplo_cidade = df['bairro'].iloc[0] if 'bairro' in df.columns else 'Coluna não achada'
            exemplo_razao = df['razao_social'].iloc[0]
            print(f"   -> Sucesso! Tabela atualizada.")
            print(f"   -> Teste de Acento: '{exemplo_razao}'")

        except UnicodeDecodeError:
            print("   -> ERRO DE ENCODING: O arquivo não é UTF-8. Tente trocar para 'latin-1' no código.")
        except Exception as e:
            print(f"   -> Erro ao processar: {e}")

# --- EXECUÇÃO ---
if __name__ == "__main__":
    db_nome = 'base_ans_paralela.db'
    arquivo_csv = 'Relatorio_cadop.csv'

    if os.path.exists(arquivo_csv):
        importador = ImportadorCadop(db_path=db_nome, csv_path=arquivo_csv)
        importador.processar_cadop()
    else:
        print(f"Arquivo {arquivo_csv} não encontrado.")